In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


data.csv получен из EDA.ipynb. Добавил там маленькое окошко в середине кода

In [ ]:
data = pd.read_csv('../data/data.csv')
data.head(2)

,price,area,rooms,ceilingHeight,kitchen_space,floor,floorsTotal,creationDate,seller,nearest_metro,...,parks_count,ponds_count,nearest_pond,time_pond,distance_pond,time_airport_via_car,distance_airport,images,city,street
0,2950000,39.5,0,2.5,0.0,1,6,2025-04-04,AGENCY,Проспект Ветеранов,...,2,1,река Кристателька,288.0,394.0,2696.0,46274.0,['//avatars.mds.yandex.net/get-realty-offers/1...,петергоф,кооперативная улица
1,2900000,23.0,0,2.8,0.0,4,11,2025-02-03,PRIVATE_AGENT,Пролетарская,...,0,1,река Нева,420.0,596.0,2452.0,29744.0,['//avatars.mds.yandex.net/get-realty-offers/1...,городской посёлок имени свердлова,западный проезд


В этом бейзлайне будет тестироваться модель без картинок


In [5]:
data = data.drop('images', axis=1)
data.head(2)

,price,area,rooms,ceilingHeight,kitchen_space,floor,floorsTotal,creationDate,seller,nearest_metro,...,branch_metro_color,parks_count,ponds_count,nearest_pond,time_pond,distance_pond,time_airport_via_car,distance_airport,city,street
0,2950000,39.5,0,2.5,0.0,1,6,2025-04-04,AGENCY,Проспект Ветеранов,...,f03d2f,2,1,река Кристателька,288.0,394.0,2696.0,46274.0,петергоф,кооперативная улица
1,2900000,23.0,0,2.8,0.0,4,11,2025-02-03,PRIVATE_AGENT,Пролетарская,...,23a12c,0,1,река Нева,420.0,596.0,2452.0,29744.0,городской посёлок имени свердлова,западный проезд


In [6]:
data.dtypes

price                     int64
area                    float64
rooms                     int64
ceilingHeight           float64
kitchen_space           float64
floor                     int64
floorsTotal               int64
creationDate             object
seller                   object
nearest_metro            object
time_to_metro           float64
transport_to_metro       object
branch_metro_color       object
parks_count               int64
ponds_count               int64
nearest_pond             object
time_pond               float64
distance_pond           float64
time_airport_via_car    float64
distance_airport        float64
city                     object
street                   object
dtype: object

In [7]:
categorical_features = [
    'seller', 'nearest_metro', 'transport_to_metro',
    'branch_metro_color', 'city', 'street', 'nearest_pond'
]

data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44440 entries, 0 to 44439
Columns: 1639 entries, price to nearest_pond_река Юнтоловка
dtypes: bool(1624), float64(8), int64(6), object(1)
memory usage: 73.9+ MB


In [8]:
data['creationDate'] = pd.to_datetime(data['creationDate'])
data['creationDate_ts'] = data['creationDate'].astype(np.int64) // 10**9

X = data.drop(['price', 'creationDate'], axis=1)
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Обучающая выборка: {X_train.shape}, Тестовая выборка: {X_test.shape}')

Обучающая выборка: (35552, 1638), Тестовая выборка: (8888, 1638)


In [9]:
X.head(2)

,area,rooms,ceilingHeight,kitchen_space,floor,floorsTotal,time_to_metro,parks_count,ponds_count,time_pond,...,nearest_pond_река Тызьва,nearest_pond_река Утка,nearest_pond_река Фонтанка,nearest_pond_река Чернавка,nearest_pond_река Чухонка,nearest_pond_река Чёрная,nearest_pond_река Чёрная Речка,nearest_pond_река Шингарка,nearest_pond_река Юнтоловка,creationDate_ts
0,39.5,0,2.5,0.0,1,6,50.0,2,1,288.0,...,False,False,False,False,False,False,False,False,False,1743724800
1,23.0,0,2.8,0.0,4,11,41.0,0,1,420.0,...,False,False,False,False,False,False,False,False,False,1738540800


## Посмотрим, что нам скажет предсказание константой


In [13]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error as MAPE

In [15]:
median_value = y_train.median()

y_pred_median = np.full(shape=y_test.shape, fill_value=median_value)

rmse_median = np.sqrt(mean_squared_error(y_test, y_pred_median))
mae_median = mean_absolute_error(y_test, y_pred_median)
r2_score_median = r2_score(y_test, y_pred_median)
MAPE_median = MAPE(y_test, y_pred_median)

print('Медиана:')
print(f'Median value: {median_value:.2f} руб.')
print(f'RMSE: {rmse_median:.2f} руб.')
print(f'MAE: {mae_median:.2f} руб.')
print(f'R2_Score: {r2_score_median:.2f}')
print(f'MAPE: {MAPE_median:.2f}')

Медиана:
Median value: 10000000.00 руб.
RMSE: 23652338.60 руб.
MAE: 7848729.44 руб.
R2_Score: -0.04
MAPE: 0.48


In [16]:
mean_value = y_train.mean()

y_pred_mean = np.full(shape=y_test.shape, fill_value=mean_value)

rmse_mean = np.sqrt(mean_squared_error(y_test, y_pred_mean))
mae_mean = mean_absolute_error(y_test, y_pred_mean)
r2_score_mean = r2_score(y_test, y_pred_mean)
MAPE_mean = MAPE(y_test, y_pred_mean)

print('Среднее:')
print(f'Mean value: {mean_value:.2f} руб.')
print(f'RMSE: {rmse_mean:.2f} руб.')
print(f'MAE: {mae_mean:.2f} руб.')
print(f'R2_Score: {r2_score_mean:.2f}')
print(f'MAPE: {MAPE_mean:.2f}')

Среднее:
Mean value: 14791650.07 руб.
RMSE: 23177765.38 руб.
MAE: 8951503.74 руб.
R2_Score: -0.00
MAPE: 0.81


## Итак, линейная регрессия

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)

rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_score_lr = r2_score(y_test, y_pred_lr)
MAPE_lr = MAPE(y_test, y_pred_lr)

print('Линейная регрессия:')
print(f'RMSE: {rmse_lr:.2f} руб.')
print(f'MAE: {mae_lr:.2f} руб.')
print(f'R2_Score: {r2_score_lr:.2f}')
print(f'MAPE: {MAPE_lr:.2f}')

Линейная регрессия:
RMSE: 45171426.09 руб.
MAE: 4478130.85 руб.
R2_Score: -2.80
MAPE: 0.30


Пока писал код, подумал, что разделение нечестное - вдруг в обучающей выборке однокомнатных мало. Нарушено процентное соотношение

Из-за этого например выше, при get_dummies, я пощадил rooms колонку

Исправим

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=X['rooms']
)

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)

rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_score_lr = r2_score(y_test, y_pred_lr)
MAPE_lr = MAPE(y_test, y_pred_lr)

print('Линейная регрессия:')
print(f'RMSE: {rmse_lr:.2f} руб.')
print(f'MAE: {mae_lr:.2f} руб.')
print(f'R2_Score: {r2_score_lr:.2f}')
print(f'MAPE: {MAPE_lr:.2f}')

Линейная регрессия:
RMSE: 11588783.34 руб.
MAE: 3863275.96 руб.
R2_Score: 0.71
MAPE: 0.28


Улучшилось - MAE немного вырос, но RMSE сильно понизился, R2 улучшился разительно

Попробуем в нормализацию и Elastic net

In [20]:
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [30]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('elasticnet', ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42))
])

In [31]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('elasticnet', ElasticNet(alpha=0.1, random_state=42))])

In [32]:
y_pred = pipeline.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2_score_sc = r2_score(y_test, y_pred)
MAPE_sc = MAPE(y_test, y_pred)

print("ElasticNet с StandardScaler:")
print(f'RMSE: {rmse:.2f} руб.')
print(f'MAE: {mae:.2f} руб.')
print(f'R2_Score: {r2_score_sc:.2f}')
print(f'MAPE: {MAPE_sc:.2f}')

ElasticNet с StandardScaler:
RMSE: 11946967.94 руб.
MAE: 3341215.23 руб.
R2_Score: 0.69
MAPE: 0.23


Улучшение есть, но не самый сок - время экспериментов с параметрами

Использую GridSearchCV = перебор + cross validation

In [25]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('elasticnet', ElasticNet(random_state=42))
])

In [26]:
param_grid = {
    'elasticnet__alpha': [0.001, 0.01, 0.1, 1, 10],
    'elasticnet__l1_ratio': [0.1, 0.5, 0.7, 0.9, 1.0]
}

In [34]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=5,
    verbose=0
)

In [35]:
grid_search.fit(X_train, y_train)

/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)
/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)
/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)
/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)
/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


KeyboardInterrupt: 

In [ ]:
print("Лучшие параметры:", grid_search.best_params_)

Лучшие параметры: {'elasticnet__alpha': 0.001, 'elasticnet__l1_ratio': 0.5}


In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2_score_sc = r2_score(y_test, y_pred)
MAPE_sc = MAPE(y_test, y_pred)

print(f'RMSE: {rmse:.2f} руб.')
print(f'MAE: {mae:.2f} руб.')
print(f'R2_Score: {r2_score_sc:.2f}')
print(f'MAPE: {MAPE_sc:.2f}')


Тестовая RMSE: 9379893.11 руб.
Тестовая MAE: 3630052.61 руб.


Неоднозначный результат - MAE увеличилось, но RMSE улучшилось

Попробуем другую сетку

In [24]:
param_grid = {
    'elasticnet__alpha': [0.001, 0.01, 0.1, 0.2, 0.3, 0.5, 0.7, 1, 3, 5, 10],
    'elasticnet__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=5,
    verbose=0
)

In [ ]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 66 candidates, totalling 330 fits


/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)
/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)
/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)
/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)
/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)
/home/dan11l/.local/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the fea

[CV] END ..elasticnet__alpha=0.001, elasticnet__l1_ratio=0.1; total time= 1.9min


/home/dan11l/.local/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.553e+18, tolerance: 1.303e+15
  model = cd_fast.enet_coordinate_descent(


[CV] END ..elasticnet__alpha=0.001, elasticnet__l1_ratio=0.1; total time= 1.9min


...

[CV] END .....elasticnet__alpha=10, elasticnet__l1_ratio=1.0; total time= 1.5min


/home/dan11l/.local/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.716e+18, tolerance: 1.360e+15
  model = cd_fast.enet_coordinate_descent(


[CV] END .....elasticnet__alpha=10, elasticnet__l1_ratio=1.0; total time= 1.5min


/home/dan11l/.local/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.070e+18, tolerance: 1.669e+15
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('elasticnet',
                                        ElasticNet(random_state=42))]),
             n_jobs=5,
             param_grid={'elasticnet__alpha': [0.001, 0.01, 0.1, 0.2, 0.3, 0.5,
                                               0.7, 1, 3, 5, 10],
                         'elasticnet__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9,
                                                  1.0]},
             scoring='neg_root_mean_squared_error', verbose=2)

In [27]:
print("Лучшие параметры:", grid_search.best_params_)

Лучшие параметры: {'elasticnet__alpha': 0.001, 'elasticnet__l1_ratio': 0.3}


In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2_score_sc = r2_score(y_test, y_pred)
MAPE_sc = MAPE(y_test, y_pred)

print(f'RMSE: {rmse:.2f} руб.')
print(f'MAE: {mae:.2f} руб.')
print(f'R2_Score: {r2_score_sc:.2f}')
print(f'MAPE: {MAPE_sc:.2f}')

RMSE: 9375013.78 руб.
MAE: 3624186.92 руб.
